# MERFISH Data reorganization

In [3]:
import pandas as pd
import numpy as np
import os, sys, time
import re
import shutil
from importlib import reload
# this version doesn't have ChromAn compiled, so directly load from src:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares/ChromAn")
import src

In [4]:
from src.file_io.data_organization import search_fovs_in_folders, Color_Usage
data_home =  r"/lab/weissman_imaging/puzheng"

data_folder = os.path.join(data_home, '4T1Tumor', '20240108-MF4_F171-4')

fds, fovs = search_fovs_in_folders(data_folder)


- searching in folder: /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4
-- 10 folders, 142 fovs detected.


In [5]:
# Color usage file marks the organization of imaging files
color_usage_filename = os.path.join(data_folder, 'Analysis', 'color_usage.csv')
color_usage_df = Color_Usage(color_usage_filename)

- load color_usage from file: /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Analysis/color_usage.csv


In [6]:
date = os.path.basename(data_folder).split('-')[0]

sample = '4T1F171'
library = 'MF4'

print(date, sample, library)

20240108 4T1F171 MF4


In [7]:
# load data_organization and match:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")
from ChromAn.src.file_io.data_organization import Data_Organization, search_fovs_in_folders
# Manually change this data_organization file as a check
data_organization_filename = r'../../merlin_parameters/dataorganization/20240108-MF4_16bit.csv'
_do = Data_Organization(data_organization_filename,)

- load color_usage from file: ../../merlin_parameters/dataorganization/20240108-MF4_16bit.csv


In [9]:
from src.file_io.data_organization import create_folder
# shutter files
ref_image_type = _do.loc[0,'imageType'] #'748_637_477_405_s13_n1000' # ref 58 / 65
polyt_image_type = _do.loc[_do['channelName']=='PolyT','imageType'].values[0] #'748_637_477_s13_n1000'
image_type = _do.loc[5,'imageType'] #'748_637_477_s13_n1000' # ref 36/39

# fiducial beads
#ref_channel = 488
#ref_frame = 13 # start from 0!

# target folder
merfish_data_folder = r'/lab/weissman_imaging/puzheng/MERFISH_data'

# target folder:
target_folder = os.path.join(merfish_data_folder, f'{date}-{sample}_{library}')
create_folder(target_folder)
print(ref_image_type, polyt_image_type, image_type)

Folder: /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4 already exists
748_637_477_405_s13 748_637_477_s13 748_637_477_s13


In [10]:
#sel_fovs = []
folder_regexp = r'H([0-9]+)[MRCP]([0-9+])'

ref_round = 0
polyt_round = _do.loc[_do['channelName']=='PolyT','imagingRound'].values[0]
#fov_ids = np.arange(2)
overwrite = True
remove_source = False

#sel_fovs = [43]

for _fov_id, _fov_name in enumerate(fovs):
    # check if this fov is selected:
    if 'sel_fovs' in locals() and _fov_id not in sel_fovs:
        continue # skip if not selected
    # prceed for the rest:
    for _fd in fds:
        # match round and hyb
        _match = re.search(folder_regexp, os.path.basename(_fd))
        if _match is None:
            continue
        # if not exist in color_usage, skip
        if os.path.basename(_fd) not in list(color_usage_df.index):
            continue
        _round, _region = _match.groups()
        # mutliple files for each image:
        _files = [os.path.join(_fd, _fl) for _fl in os.listdir(_fd) if _fov_name.split(os.extsep)[0] in _fl]
        # reference round specified
        if int(_round) == ref_round:
            _image_type = ref_image_type
        elif int(_round) == polyt_round:
            _image_type = polyt_image_type
        else:
            _image_type = image_type
    
        # target file
        _target_files = [os.path.join(target_folder, f"{_image_type}_{_fov_id}_{_round}{os.extsep}{_fl.split(os.extsep)[-1]}") 
                         for _fl in _files]
        for _fl, _tar_fl in zip(_files, _target_files):
            if remove_source:
                print(f"move {_fl} to {_tar_fl}")
                shutil.move(_fl, _tar_fl)
            else:
                if not os.path.isfile(_tar_fl) or overwrite:
                    print(f"copy {_fl} to {_tar_fl}")
                    shutil.copyfile(_fl, _tar_fl)
                else:
                    print(f"{_tar_fl} already exist, skip")

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_000.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_0_0.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_0_0.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_000.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_0_0.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_000.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_0_0.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_0_0.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_0

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_001.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_1_0.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_001.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_1_0.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_001.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_1_1.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_001.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_1_1.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_001.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_1_1.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_001.off to /l

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_002.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_2_0.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_002.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_2_1.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_002.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_2_1.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_002.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_2_1.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_002.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_2_1.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_002.off to /lab/w

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_003.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_3_1.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_003.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_3_1.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_003.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_3_1.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_003.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_3_1.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_003.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_3_1.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_003.inf to /lab/weiss

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_004.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_4_1.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_004.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_4_2.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_004.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_4_2.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_004.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_4_2.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_004.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_4_2.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_004.power to /lab/weissma

...

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_137.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_137_0.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_137.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_137_0.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H0M1/Conv_zscan_137.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_405_s13_137_0.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_137.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_137_1.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_137.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_137_1.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_z

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_138.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_138_1.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_138.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_138_1.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_138.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_138_1.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_138.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_138_1.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_138.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_138_1.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_138.inf to 

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H1M2/Conv_zscan_139.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_139_1.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_139.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_139_2.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_139.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_139_2.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_139.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_139_2.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_139.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_139_2.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_139.power to /l

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_140.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_140_2.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_140.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_140_2.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_140.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_140_2.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_140.power to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_140_3.power
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_140.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_140_3.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_140.xml to 

copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_141.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_141_2.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H2M3/Conv_zscan_141.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_141_2.xml
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_141.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_141_3.inf
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_141.off to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_141_3.off
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_141.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20240108-4T1F171_MF4/748_637_477_s13_141_3.dax
copy /lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/H3M4/Conv_zscan_141.xml to /lab

## Check file length

In [11]:
# double check if filenumber equals to expected:
print(len(os.listdir(target_folder)), len(color_usage_df) * len(fovs) * 5)

6390 6390


## Create analysis folder

In [12]:
analysis_home = r'/lab/weissman_imaging/puzheng/MERFISH_analysis'
analysis_folder = os.path.join(analysis_home, os.path.basename(target_folder))
analysis_segmentation_folder = os.path.join(analysis_folder, 'CellPoseSegment', 'segmentation_label')

print(analysis_segmentation_folder)
if not os.path.exists(analysis_segmentation_folder):
    os.makedirs(analysis_segmentation_folder)


/lab/weissman_imaging/puzheng/MERFISH_analysis/20240108-4T1F171_MF4/CellPoseSegment/segmentation_label


## Copy Segmentation files

In [21]:
segmentation_files = [os.path.join(data_folder, 'Segmentation', _f) 
                      for _f in os.listdir(os.path.join(data_folder, 'Segmentation')) 
                      if 'segmentation_label' in _f and '.npy' in _f]
target_folder = analysis_segmentation_folder

# copy
for _f in segmentation_files:
    print(_f)
    shutil.copyfile(_f, os.path.join(target_folder, os.path.basename(_f)))
    

/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_0.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_1.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_2.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_3.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_4.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_5.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_6.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_7.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_8.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_9.npy
/lab/weissman_imaging/puzheng/

/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_84.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_85.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_86.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_87.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_88.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_89.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_90.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_91.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_92.npy
/lab/weissman_imaging/puzheng/4T1Tumor/20240108-MF4_F171-4/Segmentation/segmentation_label_93.npy
/lab/weissman_imagin

In [17]:
import shutil

In [18]:
shutil.copyfile?

# Generate data_organization

go to jupyter: https://c4b15.wi.mit.edu/user/puzheng/notebooks/lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/MERFISH_preprocess/Data_organization/20230830-Generate_dataOrganization.ipynb

In [ ]:
## Automatic generate data_organization
from src.file_io.data_organization import Color_Usage

In [ ]:
def generate_dataOrganization():
    

